In [1]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [2]:
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras import Model
import numpy as np

In [3]:
inputs = Input(shape=(5,5,3))
x = Conv2D(2, 3)(inputs)
model = Model(inputs, x)

In [4]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5, 5, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 3, 3, 2)        │            56 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56 (224.00 B)

 Trainable params: 56 (224.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
model.weights

[<KerasVariable shape=(3, 3, 3, 2), dtype=float32, path=conv2d/kernel>,
 <KerasVariable shape=(2,), dtype=float32, path=conv2d/bias>]

In [6]:
X = np.random.rand(1,5,5,3)
X

array([[[[0.7208955 , 0.64531661, 0.90751283],
         [0.42566106, 0.05332064, 0.04523928],
         [0.67060903, 0.56389441, 0.00121818],
         [0.82373168, 0.53898384, 0.47770669],
         [0.38389926, 0.48628045, 0.03947125]],

        [[0.86797433, 0.77047806, 0.37122785],
         [0.44078331, 0.2403218 , 0.64706895],
         [0.4165847 , 0.32688199, 0.53021017],
         [0.68265256, 0.95320909, 0.21957465],
         [0.80327095, 0.11622152, 0.96213291]],

        [[0.53592035, 0.05747584, 0.69511268],
         [0.09821128, 0.59834817, 0.2036347 ],
         [0.46135613, 0.06088502, 0.97836599],
         [0.59891393, 0.68825388, 0.52123245],
         [0.08042598, 0.74765607, 0.31701709]],

        [[0.83965824, 0.82036876, 0.35065467],
         [0.6843708 , 0.30596596, 0.62278576],
         [0.6325423 , 0.58789653, 0.90612288],
         [0.85282076, 0.69893665, 0.02364423],
         [0.17990578, 0.10198755, 0.10290593]],

        [[0.05837675, 0.92523174, 0.62506433],
     

In [7]:
y = model.predict(X)
y

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


array([[[[-0.12057015, -0.7433455 ],
         [ 0.18239762, -0.97681   ],
         [ 0.85711664, -0.46013656]],

        [[ 0.37365782, -0.798859  ],
         [ 0.12433352, -1.1540773 ],
         [ 0.22571237, -0.16052112]],

        [[ 0.5239856 , -0.8249015 ],
         [ 0.613822  , -0.83026105],
         [-0.08674915, -0.21036346]]]], dtype=float32)

In [8]:
X_in = [[[int(X[0][i][j][k]*2**10) for k in range(3)] for j in range(5)] for i in range(5)]
X_in_unchanged = [[[str(X[0][i][j][k]) for k in range(3)] for j in range(5)] for i in range(5)]
weights = [[[[int(model.weights[0].numpy()[i][j][k][l]*2**10) for l in range(2)] for k in range(3)] for j in range(3)] for i in range(3)]
weights_unchanged = [[[[str(model.weights[0].numpy()[i][j][k][l]) for l in range(2)] for k in range(3)] for j in range(3)] for i in range(3)]
bias = [int(model.weights[1].numpy()[i]*1e72) for i in range(2)]
bias_unchanged = [str(model.weights[1].numpy()[i]) for i in range(2)]

In [9]:
def Conv2DInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    Input = [[[str(input[i][j][k]) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[[str(weights[i][j][k][l]) for l in range(nFilters)] for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
    Bias = [str(bias[i]) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    for i in range((nRows - kernelSize)//strides + 1):
        for j in range((nCols - kernelSize)//strides + 1):
            for m in range(nFilters):
                for k in range(nChannels):
                    for x in range(kernelSize):
                        for y in range(kernelSize):
                            out[i][j][m] += input[i*strides+x][j*strides+y][k] * weights[x][y][k][m]
                out[i][j][m] += bias[m]
                out[i][j][m] = str(out[i][j][m] // n)
    return Input, Weights, Bias, out

In [10]:
X_in, weights, bias, out = Conv2DInt(5, 5, 3, 2, 3, 1, 2**10, X_in, weights, bias)
out

[[['-125', '-759'], ['187', '-998'], ['876', '-471']],
 [['381', '-817'], ['126', '-1179'], ['230', '-163']],
 [['535', '-842'], ['628', '-849'], ['-90', '-214']]]

In [11]:
in_json = {
    "in": X_in,
    "weights": weights,
    "bias": bias,
    "out": out,
    "in_unchanged": X_in_unchanged,
    "weights_unchanged": weights_unchanged,
    "bias_unchanged": bias_unchanged
}

In [12]:
import json

In [13]:
with open("conv2D_input.json", "w") as f:
    json.dump(in_json, f)